In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import os

In [ ]:

BATCH_SIZE = 32
IMAGE_SIZE = 256
CHANNELS = 3
EPOCHS = 32

In [ ]:
# Load the dataset using image_dataset_from_directory
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "beansLeaf",
    seed=123,
    shuffle=True,
    image_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE
)
class_names = dataset.class_names


In [ ]:
# Split the dataset into training, validation, and test sets
test_ds = dataset.map(lambda x, y: (x, y))

In [ ]:
# Convert dataset to NumPy arrays
images = []
labels = []
for image_batch, label_batch in dataset:
    images.append(image_batch.numpy())
    labels.append(label_batch.numpy())
images = np.concatenate(images)
labels = np.concatenate(labels)

In [ ]:
# Create an instance of the ImageDataGenerator with desired augmentation options
data_augmentation = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.1,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    vertical_flip=False,
    fill_mode='nearest'
)

In [ ]:
# Create the directory if it doesn't exist
augmented_images_dir = "augmented_images"
if not os.path.exists(augmented_images_dir):
    os.makedirs(augmented_images_dir)

In [ ]:
# Generate augmented images and save them to the directory
augmented_data_generator = data_augmentation.flow(images, labels, batch_size=BATCH_SIZE, shuffle=True)
for i, (augmented_images, augmented_labels) in enumerate(augmented_data_generator):
    for j, augmented_image in enumerate(augmented_images):
        save_path = os.path.join(augmented_images_dir, f"augmented_image_{i * BATCH_SIZE + j}.png")
        tf.keras.preprocessing.image.save_img(save_path, augmented_image)
    if (i + 1) * BATCH_SIZE >= len(images):
        break

In [ ]:

# Create a generator function for augmented image batches and labels
def augmented_data_generator():
    for image_batch, label_batch in data_augmentation.flow(images, labels, batch_size=BATCH_SIZE, shuffle=False):
        yield image_batch, label_batch

In [ ]:
# Create a new dataset from the generator function
augmented_dataset = tf.data.Dataset.from_generator(
    augmented_data_generator,
    output_signature=(
        tf.TensorSpec(shape=(BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(BATCH_SIZE,), dtype=tf.int32)
    )
)

In [ ]:
# Concatenate the augmented dataset and custom dataset
combined_dataset = augmented_dataset.concatenate(dataset)

In [ ]:

# Shuffle and batch the combined dataset
combined_dataset = combined_dataset.shuffle(len(images)).batch(BATCH_SIZE)

In [ ]:

# Split the combined dataset into training and validation sets
val_size = int(0.2 * len(images))
train_dataset = combined_dataset.skip(val_size)
val_dataset = combined_dataset.take(val_size)

In [ ]:
# Define a function to create the model with the given hyperparameters
def create_model(lr, batch_size, n_layers):
    model = models.Sequential()
    model.add(layers.experimental.preprocessing.Rescaling(1./255, input_shape=(IMAGE_SIZE, IMAGE_SIZE, CHANNELS)))
    model.add(layers.Conv2D(32, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    for _ in range(n_layers):
        model.add(layers.Conv2D(64, (3, 3), activation='relu'))
        model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(len(class_names), activation='softmax'))
    model.compile(optimizer=tf.keras.optimizers.Adam(lr=lr),
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])
    return model

In [ ]:
learning_rates = [0.001, 0.01, 0.1]
batch_sizes = [16, 32, 64]
num_layers = [2, 3, 4]

best_accuracy = 0
best_model = None

for lr in learning_rates:
    for batch_size in batch_sizes:
        for n_layers in num_layers:
            model = create_model(lr, batch_size, n_layers)
            history = model.fit(train_dataset, validation_data=val_dataset, epochs=EPOCHS)
            accuracy = history.history['val_accuracy'][-1]
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_model = model

In [ ]:
# Evaluate the best model on the test set
test_loss, test_accuracy = best_model.evaluate(test_ds)

In [ ]:


# Save the best model
best_model.save("../beansmodelhyper.h5")